# Debugging

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas/main?filepath=tutorials/0-debug.ipynb)

Minimal working examples of bugs

**<p style="color:#e31883">This notebook demonstrates the current development version of MovingPandas.</p>**

For tutorials using the latest release visit https://github.com/movingpandas/movingpandas-examples.


In [2]:
import urllib
import os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

import sys

sys.path.append("..")
import movingpandas as mpd

mpd.show_versions()


MovingPandas 0.22.3

SYSTEM INFO
-----------
python     : 3.10.15 | packaged by conda-forge | (main, Oct 16 2024, 01:24:24) [GCC 13.3.0]
executable : /home/anita/miniforge3/envs/movingpandas-dev/bin/python
machine    : Linux-6.8.0-59-generic-x86_64-with-glibc2.39

PROJ INFO
-----------
PROJ       : 9.5.0
PROJ data dir: /home/anita/miniforge3/envs/movingpandas-dev/share/proj

PYTHON DEPENDENCIES
-------------------
numpy      : 2.0.2
geopandas  : 1.0.1
geopy      : 2.4.1
geoviews   : 1.13.0
holoviews  : 1.19.1
hvplot     : 0.11.1
mapclassify: 2.8.1
matplotlib : 3.9.2
pandas     : 2.2.3
pyproj     : 3.7.0
shapely    : 2.0.6
stonesoup  : 1.4


## Creating a trajectory from scratch


In [ ]:
df = pd.DataFrame(
    [
        {"geometry": Point(0, 0), "t": datetime(2018, 1, 1, 12, 0, 0)},
        {"geometry": Point(6, 0), "t": datetime(2018, 1, 1, 12, 6, 0)},
        {"geometry": Point(6, 6), "t": datetime(2018, 1, 1, 12, 10, 0)},
        {"geometry": Point(9, 9), "t": datetime(2018, 1, 1, 12, 15, 0)},
    ]
).set_index("t")
gdf = GeoDataFrame(df, crs=31256)
toy_traj = mpd.Trajectory(gdf, 1)
toy_traj.df

In [ ]:
df = pd.DataFrame(
    [
        {"geometry": Point(0, 0), "t": datetime(2018, 1, 1, 12, 0, 0)},
        {"geometry": Point(6, 0), "t": datetime(2018, 1, 1, 12, 6, 0)},
        {"geometry": Point(6, 6), "t": datetime(2018, 1, 1, 12, 10, 0)},
        {"geometry": Point(9, 9), "t": datetime(2018, 1, 1, 12, 15, 0)},
    ]
).set_index("t")
gdf = GeoDataFrame(df, crs=31256)
df = pd.DataFrame(gdf)
df['x']=gdf.geometry.x
df['y']=gdf.geometry.y
df = df.drop(columns='geometry')
toy_traj = mpd.Trajectory(df, 1, x="x", y="y")
toy_traj.df

In [ ]:
toy_traj.hvplot()

In [ ]:
toy_traj.add_speed(overwrite=True).df

In [ ]:
xmin, xmax, ymin, ymax = 2, 8, -10, 5
polygon = Polygon(
    [(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)]
)
polygon_gdf = GeoDataFrame(pd.DataFrame([{"geometry": polygon, "id": 1}]), crs=4326)

In [ ]:
intersections = toy_traj.clip(polygon)
intersections

In [ ]:
intersections = toy_traj.clip(polygon)
intersections

In [ ]:
ax = toy_traj.plot()
polygon_gdf.plot(ax=ax, color="lightgray")
intersections.plot(ax=ax, color="red", linewidth=5)

In [ ]:
toy_traj.add_speed(overwrite=True)
toy_traj.df.plot()

In [ ]:
toy_traj.plot(width=7)

In [ ]:
toy_traj.plot(column="speed", linewidth=5)

In [ ]:
toy_traj.explore()

In [ ]:
hv_defaults = {"frame_width": 400, "frame_height": 300}
toy_traj.hvplot(line_width=5, **hv_defaults)

In [ ]:
toy_traj.hvplot(c="speed", line_width=7, **hv_defaults)

In [ ]:
toy_traj.hvplot_pts(c="speed", line_width=7, **hv_defaults)

In [ ]:
gdf = read_file("data/demodata_geolife.gpkg")
tc = mpd.TrajectoryCollection(gdf, "trajectory_id", t="t")
generalized = mpd.MinDistanceGeneralizer(tc).generalize(tolerance=100)

In [ ]:
generalized.plot()

In [ ]:
generalized.explore(
    column="trajectory_id", tiles="CartoDB positron", style_kwds={"weight": 5}
)

In [ ]:
generalized.add_speed(overwrite=True).explore(
    column="speed", vmax=20, tiles="CartoDB positron", style_kwds={"weight": 5}
)

In [ ]:
generalized.hvplot(tiles="CartoLight", **hv_defaults)

In [ ]:
generalized.hvplot(
    tiles=None, c="speed", clim=(0, 20), line_width=2, marker_size=0, **hv_defaults
)

In [ ]:
feature = {
    "geometry": {
        "type": "Polygon",
        "coordinates": [[(116, 39), (116.5, 39), (116.5, 40), (116, 40), (116, 39)]],
    },
    "properties": {"id": 1, "name": "foo"},
}

In [ ]:
intersections = generalized.intersection(feature)
intersections.hvplot(
    tiles=None, c="speed", clim=(0, 20), line_width=2, marker_size=0, **hv_defaults
)

In [ ]:
# intersections.add_speed()
intersections.hvplot_pts(tiles=None, c="speed", clim=(0, 20), **hv_defaults)

In [ ]:
intersections.hvplot_pts(tiles=None, **hv_defaults)

In [ ]:
generalized.add_speed(overwrite=True).explore(
    column="speed", vmax=20, tiles="CartoDB dark_matter", style_kwds={"weight": 5}
)

In [16]:


data = {'t': {0: '2025-04-02 11:08:11.666928', 1: '2025-04-02 11:08:12.666928', 2: '2025-04-02 11:08:13.666928'}, 
        'x': {0: 10.0, 1: 10.01, 2: 10.021}, 'y': {0: 10.0, 1: 10.0, 2: 10.0}, 
        'AgentID': {0: 19, 1: 19, 2: 19}, 'wkt': {0: 'POINT (10 10)', 1: 'POINT (10.01 10)', 2: 'POINT (10.020999999999999 10)'}}

gdf = gpd.GeoDataFrame(data, crs=None)
gdf.set_geometry(gpd.GeoSeries.from_wkt(gdf['wkt']), inplace=True)
mpd.TrajectoryCollection(gdf, traj_id_col='AgentID', obj_id_col='AgentID', t='t')




/home/anita/Documents/Github/movingpandas/tutorials/../movingpandas/trajectory.py:186: MissingCRSWarning: Trajectory generated without CRS. Computations will use Euclidean distances.
  warnings.warn(


TrajectoryCollection with 1 trajectories